# Section G - Working with Databasese

Feedback: https://forms.gle/Le3RAsMEcYqEyswEA

**Topics**:
* Basics
* Sqlite
  * Pandas DB Functions
* Create Table
* Drop Table
* Update Rows

There are many types of database:  
* SQl Flavors - Sqlite, MariaDB, PosgreSQL, MySQL, MS SQL, ...
* NoSQL Flavors - Mongo DB, Elastic, Redis, ...

**Why use databases?**
* In a business enveronment, a database hosted on a server can be shared by multiple people with updates and queries happenning in parallel.  Databases are disigned to host and interacte with large volumes of data and can provide baked-in solutions like [ACID transactions](https://www.mongodb.com/resources/basics/databases/acid-transactions#what-are-acid-transactions).
* You may need to save your own data when not working with it and storing it in sqlite is one of many ways to serialize it and save it in a file.  Pandas dataframes can contain datasets so large they cannot be exported as xlxs (or csv?) files.  

## Basics
To work with a database from python, we do a couple of things:
* import database library
* Create a connection to the database - this could be a file path for sqlite, or a server name with login and password for a hosted database like Postgres. 
* Create a cursor from the connection - The cursor is what we use to run commands, inserts, updates, queries and retreive results.  
* Alternatively, pass the connection to a tool like pandas and let it generate queries/updates/inserts for you. 

A couple examples:

**Sqlite**

    import sqlite3
    
    with sqlite3.connect('example.db') as conn:
        cursor = conn.cursor()
        cursor.execute('SELECT name FROM sqlite_master WHERE type="table";')
        print(cursor.fetchall())

**Postgres**

    import psycopg2

    with psycopg2.connect(
        dbname="your_db",
        user="your_user",
        password="your_password",
        host="localhost",
        port="5432"
    ) as conn:
        with conn.cursor() as cursor:
            cursor.execute("SELECT * FROM your_table LIMIT 5;")
            print(cursor.fetchall())



## Mathematical Operations
These are the basic mathematical functoins that are included in python without importing any libraries.

In [1]:
import pandas as pd
import sqlite3
iris_web = pd.read_csv('https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv')

print(iris_web.head())

# Save the DataFrame to a SQLite database
with sqlite3.connect('iris_web.db') as conn:
    iris_web.to_sql('iris', conn, if_exists='replace', index=False)


   sepal_length  sepal_width  petal_length  petal_width species
0           5.1          3.5           1.4          0.2  setosa
1           4.9          3.0           1.4          0.2  setosa
2           4.7          3.2           1.3          0.2  setosa
3           4.6          3.1           1.5          0.2  setosa
4           5.0          3.6           1.4          0.2  setosa


In [5]:
import pandas as pd

with sqlite3.connect('iris_web.db') as conn:
    query = "SELECT * FROM iris"
    df = pd.read_sql_query(query, conn)

print(df.head())

# Without the "with" statemen:
# conn = sqlite3.connect('your_database.db')
# query = "SELECT * FROM your_table"
# df = pd.read_sql_query(query, conn)
# conn.close()

   sepal_length  sepal_width  petal_length  petal_width species
0           5.1          3.5           1.4          0.2  setosa
1           4.9          3.0           1.4          0.2  setosa
2           4.7          3.2           1.3          0.2  setosa
3           4.6          3.1           1.5          0.2  setosa
4           5.0          3.6           1.4          0.2  setosa


In [ ]:
# Without pandas, query the database directly
with sqlite3.connect('iris_web.db') as conn:
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM iris")
    rows = cursor.fetchall()
for row in rows[:10]:
    print(row)

(5.1, 3.5, 1.4, 0.2, 'setosa')
(4.9, 3.0, 1.4, 0.2, 'setosa')
(4.7, 3.2, 1.3, 0.2, 'setosa')
(4.6, 3.1, 1.5, 0.2, 'setosa')
(5.0, 3.6, 1.4, 0.2, 'setosa')
(5.4, 3.9, 1.7, 0.4, 'setosa')
(4.6, 3.4, 1.4, 0.3, 'setosa')
(5.0, 3.4, 1.5, 0.2, 'setosa')
(4.4, 2.9, 1.4, 0.2, 'setosa')
(4.9, 3.1, 1.5, 0.1, 'setosa')


In [10]:
# Update 2nd row in the df and update it in the database
df.loc[1, 'sepal_length'] = 99999
df.head()

with sqlite3.connect('iris_web.db') as conn:
    df.to_sql('iris', conn, if_exists='replace', index=False)
# Verify the update
with sqlite3.connect('iris_web.db') as conn:
    query = "SELECT * FROM iris"
    updated_df = pd.read_sql_query(query, conn)